In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Define a model client. You can use other model client that implements
# the `ChatCompletionClient` interface.
model_client = OpenAIChatCompletionClient(model="gpt-4o")


# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


In [2]:
# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


In [3]:
# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- user ----------
What is the weather in New York?
---------- weather_agent ----------
[FunctionCall(id='call_Z1bp4o7ip9wG7UPzi0aqFQjB', arguments='{"city":"New York"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_Z1bp4o7ip9wG7UPzi0aqFQjB', is_error=False)]
---------- weather_agent ----------
The current weather in New York is 73°F and sunny.


In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

# Get the fetch tool from mcp-server-fetch.
fetch_mcp_server = StdioServerParams(command="uvx", args=["mcp-server-fetch"])
fetcher = await mcp_server_tools(fetch_mcp_server)

# Create an agent that can use the fetch tool.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(name="fetcher", model_client=model_client, tools=fetcher, reflect_on_tool_use=True)  # type: ignore

# Let the agent fetch the content of a URL and summarize it.
result = await agent.run(task="Summarize the content of https://en.wikipedia.org/wiki/Autonomous_agent")
print(result.messages[-1].content)

The Wikipedia page on "Autonomous Agent" provides an overview of what constitutes an autonomous agent in the context of artificial intelligence (AI). It defines autonomous agents as systems capable of performing complex tasks independently. Various definitions include:

1. Brustoloni (1991) describes them as systems capable of purposeful action in the real world.
2. Maes (1995) states they are computational systems that can sense and act autonomously in a complex, dynamic environment towards specific goals.
3. Franklin and Graesser (1997) define them as systems situated within their environment, capable of sensing and acting over time to pursue their agenda.

Additionally, the page discusses how the design and appearance of autonomous vehicles may influence human trust and perception, highlighting the relationship between appearance, autonomy, and trustworthiness.

The article also lists related concepts and terms such as intelligent agents, multi-agent systems, and software agents.

T

In [18]:
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool


prompt = """I want to buy a one-way non-stop flight from JFK to LHR on June 1st 2025.
Please search online for any promising deals, and respond with your one good suggestion for airline name, website and price that I should do more research on."""

serper = GoogleSerperAPIWrapper()
langchain_tool =Tool(
        name="internet_search",
        func=serper.run,
        description="useful for when you need to search the internet"
    )
tool = LangChainToolAdapter(langchain_tool)

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(name="searcher", model_client=model_client, tools=[tool], reflect_on_tool_use=True)
result = await agent.run(task=prompt)
print(result.messages[-1].content)

One good suggestion for your flight from JFK to LHR on June 1, 2025, is with British Airways. 

- **Airline Name:** British Airways
- **Website:** [British Airways](https://www.britishairways.com)
- **Price:** Starting at $500

I recommend visiting their website for more details and to check for any current promotions or specific flight options. 

TERMINATE


In [19]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient


# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    tools=[tool],
    system_message="You are a helpful AI research assistant who looks for promising deals on flights. Repond only with one option, \
strictly 1 option, do not mention others that you considered. Make sure it's different from any previous option mentioned.",
)

# Create the critic agent.
evaluation_agent = AssistantAgent(
    "evaluator",
    model_client=model_client,
    system_message="Check whether this looks like a very promising recommendation. \
        Respond with 'APPROVE' when you are satisfied that the recommendation is good. \
        Make sure you've seen at least 2 options before approving.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, evaluation_agent], termination_condition=text_termination)


In [20]:
result = await team.run(task=prompt)
for message in result.messages:
    print(message.content)


I want to buy a one-way non-stop flight from JFK to LHR on June 1st 2025.
Please search online for any promising deals, and respond with your one good suggestion for airline name, website and price that I should do more research on.
[FunctionCall(id='call_2XAZ1ohcve2tUgEVMn3yZr3E', arguments='{"query":"one-way non-stop flight from JFK to LHR June 1 2025 deals"}', name='internet_search')]
[FunctionExecutionResult(content='Cheap Flights from New York (JFK) to London (LHR) start at $218 for one-way and $439 for round trip. Earn your airline miles on top of our rewards! Discover flight offers to London with Virgin Atlantic. With fares from $500, we offer a great choice of food, drinks and onboard entertainment & WiFi. Find flights to London Heathrow Airport from $52. Fly from New York John F Kennedy Airport on SWISS, Icelandair, United Airlines and more. Cheapest one-way flight, $129. Norse Atlantic UKNonstop7 hrApr 2. The cheapest one-way flight from New York to London is currently $129. 

In [22]:
from dataclasses import dataclass

from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler


@dataclass
class MyMessageType:
    content: str


class MyAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("MyAgent")

    @message_handler
    async def handle_my_message_type(self, message: MyMessageType, ctx: MessageContext) -> None:
        print(f"{self.id.type} received message: {message.content}")

In [23]:
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient


class MyAssistant(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: MyMessageType, ctx: MessageContext) -> None:
        print(f"{self.id.type} received message: {message.content}")
        response = await self._delegate.on_messages(
            [TextMessage(content=message.content, source="user")], ctx.cancellation_token
        )
        print(f"{self.id.type} responded: {response.chat_message.content}")

In [24]:
from autogen_core import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await MyAgent.register(runtime, "my_agent", lambda: MyAgent())
await MyAssistant.register(runtime, "my_assistant", lambda: MyAssistant("my_assistant"))


AgentType(type='my_assistant')

In [25]:
runtime.start()  # Start processing messages in the background.
await runtime.send_message(MyMessageType("Hello, World!"), AgentId("my_agent", "default"))
await runtime.send_message(MyMessageType("Hello, World!"), AgentId("my_assistant", "default"))
await runtime.stop()  # Stop processing messages in the background.

my_agent received message: Hello, World!
my_assistant received message: Hello, World!
my_assistant responded: Hello! How can I assist you today?
